In [3]:
!pip install torch torchvision torchaudio pandas numpy
!pip install ultralytics
!pip install tqdm

In [4]:
import cv2
import os
import numpy as np
import json
import tqdm

def get_all_image_files(root_folder):
    # Các định dạng ảnh thường gặp
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff','.JPG'}
    image_files = []
    
    # Sử dụng os.walk để đệ quy qua tất cả các thư mục và file
    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            # Kiểm tra xem file có phải là file ảnh không
            if os.path.splitext(filename)[1].lower() in image_extensions:
                image_files.append(os.path.join(dirpath, filename))
    return image_files

def predict(chosen_model, img, classes=[], conf=0.5):
    if classes:
        results = chosen_model.predict(img, classes=classes, conf=conf)
    else:
        results = chosen_model.predict(img, conf=conf)

    return results

def predict_and_detect(chosen_model, img, classes=[], conf=0.5, rectangle_thickness=2, text_thickness=1):
    results = predict(chosen_model, img, classes, conf=conf)
    for result in results:
        for box in result.boxes:
            cv2.rectangle(img, (int(box.xyxy[0][0]), int(box.xyxy[0][1])),
                          (int(box.xyxy[0][2]), int(box.xyxy[0][3])), (255, 0, 0), rectangle_thickness)
            cv2.putText(img, f"{result.names[int(box.cls[0])]}",
                        (int(box.xyxy[0][0]), int(box.xyxy[0][1]) - 10),
                        cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), text_thickness)
    return img, results

In [ ]:
import cv2
import os
import numpy as np
import json
import tqdm

def get_all_image_files(root_folder):
    # Các định dạng ảnh thường gặp
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff','.JPG'}
    image_files = []
    
    # Sử dụng os.walk để đệ quy qua tất cả các thư mục và file
    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            # Kiểm tra xem file có phải là file ảnh không
            if os.path.splitext(filename)[1].lower() in image_extensions:
                image_files.append(os.path.join(dirpath, filename))
    return image_files

def predict(chosen_model, img, classes=[], conf=0.5):
    if classes:
        results = chosen_model.predict(img, classes=classes, conf=conf)
    else:
        results = chosen_model.predict(img, conf=conf)

    return results

def predict_and_detect(chosen_model, img, classes=[], conf=0.5, rectangle_thickness=2, text_thickness=1):
    results = predict(chosen_model, img, classes, conf=conf)
    for result in results:
        for box in result.boxes:
            cv2.rectangle(img, (int(box.xyxy[0][0]), int(box.xyxy[0][1])),
                          (int(box.xyxy[0][2]), int(box.xyxy[0][3])), (255, 0, 0), rectangle_thickness)
            cv2.putText(img, f"{result.names[int(box.cls[0])]}",
                        (int(box.xyxy[0][0]), int(box.xyxy[0][1]) - 10),
                        cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), text_thickness)
    return img, results

def read_file_to_array(filename):
    with open(filename, 'r') as file:
        array = file.readlines()
    # Remove newline characters
    array = [line.strip() for line in array]
    return array


def detect(img_path,model):
    image=cv2.imread(img_path)
    inferencer=predict(model,image,[],0.4)
    inferencer=inferencer[0]
    h,w=inferencer.orig_shape
    bb=inferencer.boxes.xyxy.clone().to("cpu")
    labels=inferencer.boxes.cls.clone().to("cpu")
    labels=labels.long()
    
    return img_path,w,h,bb,labels

from collections import Counter

# def count_labels(label_list):
#     # Sử dụng Counter để đếm số lượng xuất hiện của mỗi label
    
#     label_counts = Counter(label_list)
#     return label_counts
def count_labels(labelss):
    label_count = {}
    labels=labelss.cpu().numpy()
    for label in labels:
        if label in label_count:
            label_count[label] += 1
        else:
            label_count[label] = 1
    # print(label_count)
    return label_count


def filter_oj(img_path,w,h,bb, labels):
    array=np.zeros((80,7,7))
    step_w=(w+1)/7
    step_h=(h+1)/7 
    for i in range(len(labels)):
        bb[i][0]=int(bb[i][0]/step_w)
        bb[i][1]=int(bb[i][1]/step_h)
        bb[i][2]=int(bb[i][2]/step_w)
        bb[i][3]=int(bb[i][3]/step_h)
        for y in range(int(bb[i][0]),int(bb[i][2]+1)):
            for x in range(int(bb[i][1]),int(bb[i][3]+1)):
                array[labels[i]][x][y]=1
    return array

def change_cordinate(array):
    labels = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic_light', 'fire_hydrant', 'stop_sign', 'parking_meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports_ball', 'kite', 'baseball_bat', 'baseball_glove', 'skateboard', 'surfboard', 'tennis_racket', 'bottle', 'wine_glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot_dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted_plant', 'bed', 'dining_table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell_phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy_bear', 'hair_drier', 'toothbrush']
    list=['a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'b0', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'd0', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'e0', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'g0', 'g1', 'g2', 'g3', 'g4', 'g5', 'g6']
    policy=np.zeros(49)
    last=""
    for x in range(7):
        for y in range(7):
            for i in range(80):
                if array[i][x][y]==1:
                    pos=y*7+x
                    policy[pos]=1
                    list[pos]=list[pos]+ " "+str(labels[i])
    for i in range(49):
        if policy[i]==1:
            # Tách chuỗi thành các phần tử
            parts = list[i].split()
            base = parts[0]
            for part in parts[1:]:
                last=last+base+part+" "
    return last


# def mk_json(img_paths, lists, file_name):
#     array=""
#     for i in range(len(lists)):
#         array=str(array)+"image_path: "+str(img_paths[i])+ ", object: "+ str(lists[i])+/n
#     with open(file_name, 'w') as json_file:
#         json.dump(array, json_file, indent=4)
#     print(f'Data has been saved to {file_name}')

# def mk_json(img_paths, lists, file_name):
#     data = []
    
#     for i in range(len(img_paths)):
#         # Create a dictionary for each image path and its corresponding objects
#         entry = {
#             "image_path": img_paths[i],
#             "object": lists[i]
#         }
#         data.append(entry)
    
#     # Write the list of dictionaries to a JSON file
#     for da in data:
#         with open(file_name, 'w') as json_file:
#             json.dump(da, json_file, indent=4)
    
#     print(f'Data has been saved to {file_name}')
def mk_json(img_paths, objects, file_name, label_counts):
    labels = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic_light', 'fire_hydrant', 'stop_sign', 'parking_meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports_ball', 'kite', 'baseball_bat', 'baseball_glove', 'skateboard', 'surfboard', 'tennis_racket', 'bottle', 'wine_glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot_dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted_plant', 'bed', 'dining_table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell_phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy_bear', 'hair_drier', 'toothbrush']
    
    policy=np.zeros(49)
    with open(file_name, 'w') as json_file:
        for i in range(len(img_paths)):
            counting=""
            for label, count in label_counts[i].items():
                counting=counting+str(labels[label])+str(count)+" "
            # Create a dictionary for each image path and its corresponding object
            entry = {
                "image_path": img_paths[i],
                "object": objects[i],
                "count":counting
                
                # If objects is a list of lists
            }
            # Write the dictionary as a JSON string on a single line
            json_file.write(json.dumps(entry) + '\n')
    
    print(f'Data has been saved to {file_name}')
